In [160]:
import requests
import re
import time
import os
import urllib.request
from typing import Union
import csv
import statistics

import typing
import io
import numpy as np
from docx import Document


import camelot

import pandas as pd
%config Completer.use_jedi = False
from pathlib import Path
# import tqdm.notebook.tqdm as tqdm
from tqdm import tqdm_notebook
# from tqdm.notebook import tqdm


In [2]:


def read_docx_tables(filename, tab_id=None, **kwargs) -> pd.DataFrame:
    """
    parse table(s) fromt.columnsrd Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise
        



def rename_col(col):
    col = col.lower()
    if re.search(pattern='(фамилия|имя|фио|ф.и.о.|отчество)', string=col):
        return "ФИО"

    elif re.search(pattern='(заработная плата|cреднемесячная|зарпл.)', string=col):
        return "Зарплата"

    elif re.search(pattern='(должност[и,ь])', string=col):
        return 'Должность'

    elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическое лицо)', string=col):
        return 'Учреждение'

    
    return col


def check_if_columns_ok(cols: tuple) -> bool:
    cols = list(map(str, cols))
    cols = list(map(str.lower, cols))
    one_already_found = False 
    pattern = '(фамилия|имя|фио|ф.и.о|отчество|заработная плата|cреднемесячная|зарпл|занимаемая|должност[и,ь]|(предприяти[е,я]|учреждени[е,я]))'
    
    for col in cols:
        res = re.search(pattern=pattern, string=col)
        if res and not one_already_found:
            one_already_found = True
        elif res and one_already_found:
            return True
    
    return False


    
def detect_and_rename_headers(df: pd.DataFrame) -> pd.DataFrame:

    cols = df.columns
    res = check_if_columns_ok(cols)

    if not res:        
        for index, cell in enumerate(df[:3].itertuples()):
            res = check_if_columns_ok(cell)
            if res:
                df.columns = df.iloc[index]
                df = df[index+1:]
                break
        if not res:
            return False, df
            #raise ValueError('заголовки колонок не найдены')

    cols = df.columns
    renamed_cols = []
    for col in cols:
        renamed_cols.append(rename_col(col))

    df.columns = renamed_cols
    return True, df


def read_all_docs()->list[dict[str, Document]]:
    all_docs = []
    for doc in os.listdir(folder):
        if not doc.endswith('docx'):
            continue

        try:
            all_docs.append({'file':doc,'doc':Document(folder + doc)})
        except:
            print('не вышло--', doc)
    
    return all_docs

def get_all_tables(all_docs: list[dict]) -> list[pd.DataFrame]:
    all_tables = []
    for doc_dict in all_docs:
        tables = read_docx_tables(folder + doc_dict['file'])        
        all_tables.append(tables)

    return all_tables

#all_docs = read_all_docs()


In [ ]:
class IncorrectHeaders:
    """класс для таблиц с неопределенными заголовками"""

    @staticmethod
    def search_for_office(df: pd.DataFrame):
        # ищет названия учреждени внутри таблицы 
        pass



    def parse(self, df: pd.DataFrame) -> typing.Union[pd.DataFrame, False]:
        df = self.search_for_office(df)

        return df

    # если учреждения не нашли, 





In [66]:




folder = 'data_ids/docx/'


class DataCleaner:
    """убирает лишние данные"""

    @staticmethod
    def remove_unwanted_symbols(df):        
        # TODO: чистка всех колонок
        df = df.applymap(lambda x: str(x).replace('\n', ' '))
        return df
    
    @staticmethod
    def remove_unwanted_cells(df):
        # убирает ячейки с нумерацией
        # print('--- DataCleaner.remove_unwanted_cells ---', df.columns)
        df = df[~df['position'].astype(str).str.isdigit()]
        return df
    
    @staticmethod
    def remove_short_rows(df):
        # удаляет ряды с недостаточными данными
        # ! должно применяться после выбора норм колонок
        to_remove = []
        for tup in df.itertuples():
            res = [len(str(e)) for e in tup]
            if statistics.mean(res) > 5:
                to_remove.append(tup.Index)
        
        df.drop(to_remove, inplace=True)
        return df
             

    def clean_df(self, df):
        df = self.remove_unwanted_symbols(df)
        df = self.remove_unwanted_cells(df)
        df = self.remove_short_rows(df)

        return df



class CorrectHeadersParser:

    '''класс для парсинга таблиц, у которых на месте колонки, которые нам нужны'''

    def table_splitter(self, table: pd.DataFrame, file_name) -> list[pd.DataFrame]:
        '''разделяет таблицы, в которых учреждение указано внутри таблицы'''

        def check_if_same(my_array: list) -> bool:
            
            '''проверяем одинаковые ли колонки'''

            first = my_array[0]
            for e in my_array[1:]:
                if e != first:
                    return False
            return True

        def get_indexes_to_split(table):
            index_to_split = []
            for e in range(len(table)):
                cols = table.iloc[e,:].values
                if check_if_same(cols):
                    index_to_split.append(e)
            return index_to_split


        def split_table(table: pd.DataFrame, index_to_split:Union[int, list[int]], file_name) -> list[pd.DataFrame]:
            dfs = np.array_split(table, index_to_split)
            dfs = [e for e in dfs if len(e) > 0]

            result_dfs = []
            for df in dfs:
                office = df.iloc[0,:][0]
                df = df.iloc[1:,:] 
                df['office'] = office
                result_dfs.append(df)
            
            result_dfs = [e for e in result_dfs if not e.empty]
            try:
                result_dfs = pd.concat(result_dfs)
                return result_dfs
            except Exception as ex:
                print(ex)
                print('rogue file---', file_name)
                
        index_to_split = get_indexes_to_split(table)

        if not index_to_split:
            return table

        splitted_dfs = split_table(table, index_to_split, file_name)
        return splitted_dfs

        
    def concat_name(self, df: pd.DataFrame) -> pd.DataFrame:
        '''соединяем колонки ФИО, если они в разных'''
        
        if 'name' not in df.columns:
            return df
        
        names_df = df['name']
    
        if isinstance(names_df, str) or isinstance(names_df, pd.Series):
            return df  
    
        # TODO:
        # дропнуть маленькую колонку


        names = [' '.join(e) for e in names_df.values]     
        
        df.drop(columns=['name'], inplace=True)
        df['name'] = names
        return df


    def parse(self, table: pd.DataFrame, file_name) -> pd.DataFrame:
        table = self.concat_name(table)
        table = self.table_splitter(table, file_name)
        return table



class DocxParser:

    def __init__(self):
       self.cols_we_need = ['name','salary', 'position', 'department']
       self.parse_correct_headers = CorrectHeadersParser()
       self.parse_incorrect_headers = ''
       self.all_docs: list[dict[str, Document]]
       self.data_cleaner = DataCleaner()
       


        
    def read_docx_tables(self, filename, tab_id=None, **kwargs) -> pd.DataFrame:
        """
        parse table(s) fromt.columnsrd Document (.docx) into Pandas DataFrame(s)

        Parameters:
            filename:   file name of a Word Document

            tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                        When [None] - return a list of DataFrames (parse all tables)

            kwargs:     arguments to pass to `pd.read_csv()` function

        Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
        """
        def read_docx_tab(tab, **kwargs):
            vf = io.StringIO()
            writer = csv.writer(vf)
            for row in tab.rows:
                writer.writerow(cell.text for cell in row.cells)
            vf.seek(0)
            return pd.read_csv(vf, **kwargs)

        doc = Document(filename)
        if tab_id is None:
            return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
        else:
            try:
                return read_docx_tab(doc.tables[tab_id], **kwargs)
            except IndexError:
                print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
                raise
            
    
    @staticmethod
    def rename_col(col: str) -> str:

        print('col before rename cols --', col)
        col = col.lower()
        if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col):
            return "name"

        elif re.search(pattern='(cреднемесячная|зарпл.|плат[ы, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col):
            return "salary"

        elif re.search(pattern='(должност[ь, и, ей])', string=col): 

            return 'position'

        elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическое лицо)', string=col):
            return 'department'

        return col


    @staticmethod
    def check_if_columns_ok(cols: tuple) -> bool:
        '''проверяем, есть ли в заголовках таблицы название предприятия и другая инфа'''
        
        cols = list(map(str, cols))
        cols = list(map(str.lower, cols))
        print('зашли в проверку колонок ---', cols)
        ok_cols = 0
        company_found = False
        for col in cols:
            company_pattern = '(предприяти[е,я]|учреждени[е,я]|юридическ[ое,ие])'
            res = re.search(pattern=company_pattern, string=col)
            if res:
                company_found = True
                continue
            
            name_salary_position_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|плат[ы, а]|заработная|плата|cреднемесячн[ая, ой]|зарплат[а, ной, ы]|должность|)'
            
            res = re.search(pattern=name_salary_position_pattern, string=col)
            if res:
                ok_cols+=1

        if company_found and ok_cols > 1:
            return True
        return False


    def get_all_tables_of_a_doc(self, filename: str) -> list[pd.DataFrame]:

        tables = self.read_docx_tables(filename)        
        return tables



    def parse_doc(self, filename: str) -> pd.DataFrame:
        assert filename.endswith('docx'), 'Формат должен быть .docx!'
            
        doc = Document(filename)
        # тут взять текст, который потом прикрутить к

        doc_tables = self.get_all_tables_of_a_doc(filename) 
        parsed_tables = []
        i=0
        for table in doc_tables: # в этом моменте мы отдаем df 
            i+=1
            # print('table---', table.columns)
            # table.to_excel(f'{i}_промежуточны_странный.xlsx')
            
            columns_ok = self.check_if_columns_ok(table)
            print('columns_ok----',columns_ok)
            if not columns_ok:
            # пометить?
            # если учреждения нет - смотрим параграфы. 
                # добавить документ в опущенные
                pass

            else:                
                # если заголовки ок, оставляем только нужные

                table.columns = [self.rename_col(col) for col in table.columns]

                
                cols_to_leave = [col for col in table.columns if col in self.cols_we_need]
                cols_to_leave = set(cols_to_leave)
                table = table[cols_to_leave]
                
                # проверяем на наличие вложенных таблиц и фио, разнесенных на несколько стаоблцов
                table = self.parse_correct_headers.parse(table, filename)
                # убираем лишние ячейки и символы
                table = self.data_cleaner.clean_df(table)
                parsed_tables.append(table)
                        
        parsed_tables = [e for e in parsed_tables if isinstance(e, pd.DataFrame) and not e.empty]
        
        if isinstance(parsed_tables, list):
            if parsed_tables:
                concat_tables = pd.concat(parsed_tables)
                return concat_tables
        
        elif isinstance(parsed_tables, pd.DataFrame):
            if not parsed_tables.empty:
                return concat_tables
        
        


    def parse_folder(self, path: str):
        all_tables = []
        for doc in os.listdir(path):
            pass 
        #for doc in self.all_docs:
        #     tables = self.parse_doc(doc)
        #     if tables:
        #         # return tables
        #         #print(tables)
        #         # tables = pd.concat(tables)
        #         all_tables.append({'file_name':doc, 'df':tables})
        
        # return all_tables

class Parser:
    
    def __init__(self):
        self.cols_we_need = ['name','salary', 'position', 'department']
        self.all_docs: list[str]
  
        self.docx_parser = ''    
        self.pdf_parser = ''
 
        self.parse_correct_headers = CorrectHeadersParser()     
        self.parse_incorrect_headers = ''

        self.data_cleaner = DataCleaner()
    
    
class PdfParser:

    def __init__(self):
        pass

    
    def convert_pdf_to_df(filename) -> list[pd.DataFrame]:
        tables = camelot.read_pdf(file, line_tol=2, joint_tol=10, line_scale=40, copy_text=['v'], pages='1-end') # , flavor='stream' row_tol=10
        tables = [e.df for e in tables]
        return tables



    

parser = DocxParser()
# # res = parser.parse_doc(folder + file)
# res = parser.parse_doc(folder + file)
# folder = 'data_ids/doc/'


# res = parser.parse_doc(folder + file)
# res.to_excel(folder + 'cool/ok/' + file + '.xlsx')

folder = 'data_ids/pdf/converted/'
file = '189843_2020_Rukovoditeli.docx'
file = '180480_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(lesnichestva).docx'
file = '180466_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
file = '181208_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
file = '178292_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
file = '185859_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii.docx'


file = '83327_2016_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
# doc = Document(folder + file)
# for e in doc.paragraphs:
#     print(e.text)
#     print('===')

res = parser.parse_doc(folder + file)
res


# errors = []
# for doc in tqdm_notebook(os.listdir(folder)):
#     if not doc.endswith('docx'):
#         continue
#     try:
#         res = parser.parse_doc(folder + doc)    
#         if isinstance(res, pd.DataFrame):
#             res.to_excel(folder + 'cool/' + doc + '.xlsx' )

#     except Exception as ex:
#         errors.append(doc)

зашли в проверку колонок --- ['полное наименование учреждения или\nпредприятия', 'должность', 'фио (полностью)', 'среднемесячная заработная плата\n(руб.)']
columns_ok---- True
col before rename cols -- Полное наименование учреждения или
предприятия
col before rename cols -- Должность
col before rename cols -- ФИО (полностью)
col before rename cols -- Среднемесячная заработная плата
(руб.)
зашли в проверку колонок --- ['полное наименование учреждения или\nпредприятия', 'должность', 'фио (полностью)', 'среднемесячная заработная плата\n(руб.)']
columns_ok---- True
col before rename cols -- Полное наименование учреждения или
предприятия
col before rename cols -- Должность
col before rename cols -- ФИО (полностью)
col before rename cols -- Среднемесячная заработная плата
(руб.)
зашли в проверку колонок --- ['полное наименование учреждения или\nпредприятия', 'должность', 'фио (полностью)', 'среднемесячная заработная плата\n(руб.)']
columns_ok---- True
col before rename cols -- Полное наимено

,name,position,department,salary
1,Баринов Геннадий Иванович,Начальник,Федеральное казенное учреждение «Объект № 5068...,"21282,91"
2,Субботина Любовь Михайловна,Главный бухгалтер (с 01.01.2016 по 15.08.2016),Федеральное казенное учреждение «Объект № 5068...,"25752,13"
3,Онучина Елена Витальевна,Главный бухгалтер (с 16.08.2016 по 31.12.2016),Федеральное казенное учреждение «Объект № 5068...,"18904, 74"
1,Рыбак Олег Павлович,Директор,Федеральное государственное бюджетное учрежден...,"93652,92"
2,nan,nan,Федеральное государственное бюджетное учрежден...,nan
3,nan,nan,Федеральное государственное бюджетное учрежден...,nan
4,nan,nan,Федеральное государственное бюджетное учрежден...,nan
5,Фролова Наталья Борисовна,Главный бухгалтер,Федеральное государственное бюджетное учрежден...,"32733,28"
6,nan,nan,Федеральное государственное бюджетное учрежден...,nan
7,nan,nan,Федеральное государственное бюджетное учрежден...,nan


In [54]:

# incorrect headers
# def detect_and_rename_headers(df: pd.DataFrame) -> pd.DataFrame:
#     '''!!!!   этот метод для парсинга неправильных заголовков'''
    
#     cols = df.columns
#     res = check_if_columns_ok(cols)

#     if not res:        
#         for index, cell in enumerate(df[:3].itertuples()):
#             res = check_if_columns_ok(cell)
#             if res:
#                 df.columns = df.iloc[index]
#                 df = df[index+1:]
#                 break
#         if not res:
#             return False, df

#     cols = df.columns
#     renamed_cols = []
#     for col in cols:
#         renamed_cols.append(rename_col(col))

#     df.columns = renamed_cols
#     return True, df

# TODO: есть еще ситуация - должность и учреждение в одной ячейке. можно решить доп проверкой 
# - если в должности гбуо, директор, бухгалтер - требует ручного разделения .
# TODO: добавить колонку с "требует ручной работы" 

folder = 'data_ids/docx/'


class DataCleaner:
    """убирает лишние данные"""

    @staticmethod
    def remove_unwanted_symbols(df):        
        df['name'] = df['name'].apply(lambda x: str(x).replace('\n', ' '))
        return df
    
    @staticmethod
    def remove_unwanted_cells(df):
        # убирает ячейки с нумерацией

        df = df[~df['position'].astype(str).str.isdigit()]
        return df

    def clean_df(self, df):
        df = self.remove_unwanted_symbols(df)
        df = self.remove_unwanted_cells(df)
        return df



class CorrectHeadersParser:

    '''класс для парсинга таблиц, у которых на месте колонки, которые нам нужны'''

    def table_splitter(self, table: pd.DataFrame, file_name) -> list[pd.DataFrame]:
        '''разделяет таблицы, в которых учреждение указано внутри таблицы'''

        def check_if_same(my_array: list) -> bool:
            '''проверяем одинаковые ли колонки'''

            first = my_array[0]
            for e in my_array[1:]:
                if e != first:
                    return False
            return True

        def get_indexes_to_split(table):
            index_to_split = []
            for e in range(len(table)):
                cols = table.iloc[e,:].values
                if check_if_same(cols):
                    index_to_split.append(e)
            return index_to_split


        def split_table(table: pd.DataFrame, index_to_split:Union[int, list[int]], file_name) -> list[pd.DataFrame]:
            dfs = np.array_split(table, index_to_split)
            dfs = [e for e in dfs if len(e) > 0]

            result_dfs = []
            for df in dfs:
                office = df.iloc[0,:][0]
                df = df.iloc[1:,:] 
                df['office'] = office
                result_dfs.append(df)
            
            result_dfs = [e for e in result_dfs if not e.empty]
            try:
                result_dfs = pd.concat(result_dfs)
                return result_dfs
            except Exception as ex:
                print(ex)
                print('rogue file---', file_name)
                
                


        index_to_split = get_indexes_to_split(table)

        if not index_to_split:
            return table

        splitted_dfs = split_table(table, index_to_split, file_name)
        return splitted_dfs

        
    def concat_name(self, df: pd.DataFrame) -> pd.DataFrame:
        '''соединяем колонки ФИО, если они в разных'''
        
        if 'name' not in df.columns:
            return df
        
        names_df = df['name']
        if isinstance(names_df, str) or isinstance(names_df, pd.Series):
            return df  
        
        names = [' '.join(e) for e in names_df.values]     
        
        df.drop(columns=['name'], inplace=True)
        df['name'] = names
        return df


    def parse(self, table: pd.DataFrame, file_name) -> pd.DataFrame:
        table = self.concat_name(table)
        table = self.table_splitter(table, file_name)
        return table



class DocxParser:

    def __init__(self):
       self.cols_we_need = ['name','salary', 'position', 'department']
       self.parse_correct_headers = CorrectHeadersParser()
       self.parse_incorrect_headers = ''
       self.all_docs: list[dict[str, Document]]
       self.data_cleaner = DataCleaner()

    # def read_all_docs(self, path: str)->list[dict[str, Document]]:
    #     self.path = path
    #     all_docs = []
    #     i = 0
    #     for doc in os.listdir(path)[:50]:
    #         if not doc.endswith('docx'):
    #             continue

    #         try:
    #             all_docs.append({'file':doc,'doc':Document(folder + doc)})
    #             i +=1
    #         except:
    #             print('не вышло--', doc)
        
    #     self.all_docs = all_docs
    #     print(f'нашли и загрузили {i} файлов')

        
    def read_docx_tables(self, filename, tab_id=None, **kwargs):
        """
        parse table(s) fromt.columnsrd Document (.docx) into Pandas DataFrame(s)

        Parameters:
            filename:   file name of a Word Document

            tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                        When [None] - return a list of DataFrames (parse all tables)

            kwargs:     arguments to pass to `pd.read_csv()` function

        Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
        """
        def read_docx_tab(tab, **kwargs):
            vf = io.StringIO()
            writer = csv.writer(vf)
            for row in tab.rows:
                writer.writerow(cell.text for cell in row.cells)
            vf.seek(0)
            return pd.read_csv(vf, **kwargs)

        doc = Document(filename)
        if tab_id is None:
            return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
        else:
            try:
                return read_docx_tab(doc.tables[tab_id], **kwargs)
            except IndexError:
                print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
                raise
            
    
    @staticmethod
    def rename_col(col: str) -> str:
        col = col.lower()
        if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col):
            return "name"

        elif re.search(pattern='(cреднемесячная|зарпл.|плат[ы, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col):
            return "salary"

        elif re.search(pattern='(должност[ь, и, ей])', string=col): 

            return 'position'

        elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическое лицо)', string=col):
            return 'department'

        return col


    @staticmethod
    def check_if_columns_ok(cols: tuple) -> bool:
        '''проверяем, есть ли в заголовках таблицы название предприятия и другая инфа'''
        
        cols = list(map(str, cols))
        cols = list(map(str.lower, cols))
        print('cols---', cols)
        ok_cols = 0
        company_found = False
        for col in cols:
            company_pattern = '(предприяти[е,я]|учреждени[е,я]|юридическ[ое,ие])'
            res = re.search(pattern=company_pattern, string=col)
            if res:
                company_found = True
                continue
            
            name_salary_position_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|плат[ы, а]|заработная плата|cреднемесячн[ая, ой]|зарплат[а, ной, ы]|должность|)'
            
            res = re.search(pattern=name_salary_position_pattern, string=col)
            if res:
                ok_cols+=1

        if company_found and ok_cols > 1:
            return True
        return False


    def get_all_tables_of_a_doc(self, filename: str) -> list[pd.DataFrame]:

        tables = self.read_docx_tables(filename)        
        return tables



    def parse_doc(self, filename: str) -> pd.DataFrame:
        
        assert filename.endswith('docx'), 'Формат должен быть .docx!'
            
        
        doc_tables = self.get_all_tables_of_a_doc(filename) 
        parsed_tables = []
        
        for table in doc_tables:

            # проверяем норм ли заголовки
            columns_ok = self.check_if_columns_ok(table)
            if not columns_ok:
            # если учреждения нет - смотрим параграфы. 
                # добавить документ в опущенные
                pass

            else:                
                # если заголовки ок, оставляем только нужные

                table.columns = [self.rename_col(col) for col in table.columns]
                
                cols_to_leave = [col for col in table.columns if col in self.cols_we_need]
                cols_to_leave = set(cols_to_leave)
                
                table = table[cols_to_leave]
                
                # проверяем на наличие вложенных таблиц и фио, разнесенных на несколько стаоблцов
                table = self.parse_correct_headers.parse(table, filename)
                # убираем лишние ячейки и символы
                
                
                # table = self.data_cleaner.clean_df(table)
                # parsed_tables.append(table)
                        
        parsed_tables = [e for e in parsed_tables if isinstance(e, pd.DataFrame) and not e.empty]
        
        if isinstance(parsed_tables, list):
            if parsed_tables:
                concat_tables = pd.concat(parsed_tables)
                return concat_tables
        
        elif isinstance(parsed_tables, pd.DataFrame):
            if not parsed_tables.empty:
                return concat_tables
        
        


    def parse_folder(self, path: str):
        all_tables = []
        for doc in os.listdir(path):
            pass 
        #for doc in self.all_docs:
        #     tables = self.parse_doc(doc)
        #     if tables:
        #         # return tables
        #         #print(tables)
        #         # tables = pd.concat(tables)
        #         all_tables.append({'file_name':doc, 'df':tables})
        
        # return all_tables


# file = "186956_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).docx"
# file = "102907_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).docx"
# file = '101058_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(obrazovanie).docx'
# folder = 'data_ids/docx/'

parser = DocxParser()
# res = parser.parse_doc(folder + file)
#res = parser.parse_doc(folder + file)

# df = res[0]
# df

#parser()
folder = 'data_ids/pdf/converted/'
file = '189828_2020_Rektor,_prorektory,_glavnyi_bukhgalter.docx'
file = '186642_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.docx'
file = '179984_2020_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
file = '86680_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
#file = '184284_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(obrazovanie).docx'
file = '180480_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(lesnichestva).docx'

#doc = Document(folder + file)

res = parser.parse_doc(folder + file)
res
# parser.check_if_columns_ok()

# tables = read_docx_tables(folder + file)
#len(doc.paragraphs)
# dir(doc)
# wh = doc.tables[0].table
# print(dir(wh))
# wh
# for e in doc.paragraphs:
#     print(e.text)
#     print('==')

# len(tables)

#parser.check_if_columns_ok(df.columns)
# for doc in os.listdir(folder):
#     if not doc.endswith('docx'):
#         continue
#     if 'test_rogue' in doc:

#         res = parser.parse_doc(folder + doc)
#         if isinstance(res, pd.DataFrame):
#             print(res)
            #res.to_excel(folder + 'cool/' + doc + '.xlsx' )



cols--- ['№ п/п', 'наименование учреждения', 'фио', 'замещаемая должность', 'среднемесячная заработная плата,\nруб.']


In [184]:
# len(errors)

72

In [ ]:
        
    def read_docx_tables(self, filename, tab_id=None, **kwargs):
        """
        parse table(s) fromt.columnsrd Document (.docx) into Pandas DataFrame(s)

        Parameters:
            filename:   file name of a Word Document

            tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                        When [None] - return a list of DataFrames (parse all tables)

            kwargs:     arguments to pass to `pd.read_csv()` function

        Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
        """
        def read_docx_tab(tab, **kwargs):
            vf = io.StringIO()
            writer = csv.writer(vf)
            for row in tab.rows:
                writer.writerow(cell.text for cell in row.cells)
            vf.seek(0)
            return pd.read_csv(vf, **kwargs)

        doc = Document(filename)
        if tab_id is None:
            return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
        else:
            try:
                return read_docx_tab(doc.tables[tab_id], **kwargs)
            except IndexError:
                print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
                raise
            
# file = '96461_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
# tables = read_docx_tables(folder + file)

df = tables[0]



array([False, False, False, False, False, False, False])

In [180]:
# t_df = df.applymap(lambda x: str(x).replace('h','AAAA'))
# tdf = t_df[['file', 'extension']] 
# tdf.rename(columns={'file':'name', 'extension':'name'}, inplace=True)

# tdf = tdf[tdf]
# tdf




'''1. Решить че делать с приклеиванием левой колонки к именам (мб по длине отсеить)
2. прогнать бывшие пдф
3. собрать все вместе

'''
#df.drop(df.columns[df.apply(lambda col: col.isnull().sum() > 3)], axis=1)
#t_df.drop('link', axis=1)

# df.apply(lambda x: len())
#df.applymap(lambda x: len(str(x)) )
# df.iloc[]




#eng_df.drop(columns=eng_df.iloc[:,:2].columns.tolist(), inplace=True)

# print(df.columns)
# to_drop
# df.drop()




'link'

In [99]:

errors = ["184385_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).docx",
]
errors[1]


'84706_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(ne_ukazany_FIO).docx'

In [ ]:
def check_if_columns_ok(cols: tuple) -> bool:
    '''проверяем, есть ли в заголовках таблицы название предприятия и другая инфа'''
    
    cols = list(map(str, cols))
    cols = list(map(str.lower, cols))
    
    ok_cols = 0
    company_found = False
    for col in cols:
        company_pattern = '(предприяти[е,я]|учреждени[е,я]|юридическ[ое,ие])'
        res = re.search(pattern=company_pattern, string=col)
        if res:
            company_found = True
            continue
        
        name_salary_position_pattern = '(фамилия|имя|фио|ф.и.о|отчество|заработная плата|cреднемесячная|зарпл|должность|)'
        
        res = re.search(pattern=name_salary_position_pattern, string=col)
        if res:
            ok_cols+=1

    if company_found and ok_cols > 1:
        return True
    return False

l = ['№ п/п', 'ФИО', 'Должность',
       'Среднемесячная заработная плата за 2020 год, руб.']
check_if_columns_ok(l)

False

In [ ]:
        
def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) fromt.columnsrd Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise
        
file = '96461_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx'
file = folder + file
tables = read_docx_tables(file)

In [60]:
# doc_folder = 'data_ids/doc/'
# file = '83305_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.doc'

# doc = Document(doc_folder+file)
# doc

import glob

# pattern = '*\*\*.docx'
# pattern2 = '*\*\*.doc'
# docs = glob.glob(pattern2)
# docsX = glob.glob(pattern)


from glob import glob
import re
import os
import win32com.client as win32
from win32com.client import constants

# Create list of paths to .doc files
paths = glob('D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\doc\*.doc', recursive=True)

def save_as_docx(path):
    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(path)
    doc.Activate()

    # Rename path with .docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)
    
    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)

for path in paths:
    save_as_docx(path)

#set([e[-1] for e in r])
# os.getcwd()

In [59]:

# TODO: есть еще ситуация - должность и учреждение в одной ячейке. можно решить доп проверкой 
# - если в должности гбуо, директор, бухгалтер - требует ручного разделения .
# TODO: добавить колонку с "требует ручной работы" 

'''1. допарсить, проверить
2. сформулировать вопросы
3. созвониться
4. сделать интерфейс
5. рефактор
6. пуш'''

65

In [ ]:
'''  
 объединяет несколько колонок с именем    
'''

df = pd.read_excel('data_ids/docx/fio.xlsx', engine='openpyxl')

def concat_name(df: pd.DataFrame) -> pd.DataFrame:
    names_df = df['ФИО']
    
    if isinstance(names_df, str) or isinstance(names_df, pd.Series):
        return df  

    names_df = names_df.rename(columns={'ФИО':'old_ФИО'})
    names_df['ФИО'] = names_df.apply(' '.join, axis=1)

    names_df.drop(columns='old_ФИО', inplace=True)    
    df.drop(columns='ФИО', inplace=True)
    return pd.concat([df, names_df], axis=1)




In [ ]:
# # такой парсер, который считывал бы заголовок таблицы
# # и надпись перед ним

# # если в таблицах нет колонки с учерждением смотрим инфу вокруг
# # raw = '83336_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(ne_ukazany_FIO).docx'
# # raw = folder + raw
# # # doc = folder + '83350_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(ne_ukazany_FIO).docx'
# # docx = Document(raw )
# res = []

# for doc_dict in all_docs:    
    
    
    
    
# # dfs = read_docx_tables(doc)
# #dfs[0]

ModuleNotFoundError: No module named 'fitz'